In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/IW06-07/deepxi

In [ ]:
# choose python 3.6
!update-alternatives --config python3
!apt install python3-pip
!python3 -m pip install --upgrade pip
!pip3 install deepspeech==0.1.0

In [ ]:
import wave
import numpy as np
import torch
import torchaudio
# from jiwer import compute_measures
import pickle
import os

In [ ]:
def convert(audio):
  stream = os.popen(f'deepspeech ../models/output_graph.pb {audio} ../models/alphabet.txt')
  output = stream.read()
  return output.strip()

In [ ]:
enh_path = './DeepXi/out/mhanet-1.1c/mmse-lsa'
adv_path = './DeepXi/Adversarial_Examples'
grounds = ['/long-ground.txt', '/medium-ground.txt', '/short-ground.txt']
files = ['/long-signals.txt', '/medium-signals.txt', '/short-signals.txt']

In [ ]:
trans_dict = {}
translations = []
for f, g in zip(files, grounds):
  t = open(adv_path+f, 'r')
  text = t.read().strip().split('\n')
  gr = open(adv_path+g, 'r')
  ground = gr.read().strip().split('\n')
  trans = {}
  for i in range(2, 102):
    temp = {}
    temp['base'] = ground[i-2].split('\t')[1]
    ori_trans = convert(adv_path + '/' + text[i])
    enh_ori_trans = convert(enh_path + '/' + text[i])
    # long, medium, short adv
    lms = (text[i+101], text[i+2*101], text[i+3*101])
    adv_trans = [convert(adv_path+'/'+name) for name in lms]
    enh_trans = [convert(enh_path+'/'+name) for name in lms]
    temp['ori'] = ori_trans
    temp['adv'] = adv_trans
    temp['enh'] = enh_trans
    temp['enh_ori'] = enh_ori_trans
    trans[text[i]] = temp
  with open(f'deepxi_trans1{len(translations)}.pickle', 'wb') as e:
    pickle.dump(trans, e, protocol=pickle.HIGHEST_PROTOCOL)
  translations.append(trans)
  t.close()
  gr.close()

trans_dict['long'] = translations[0]
trans_dict['medium'] = translations[1]
trans_dict['short'] = translations[2]

In [ ]:
trans_dict['long'] = translations[0]
trans_dict['medium'] = translations[1]
trans_dict['short'] = translations[2]

In [ ]:
trans_dict['short']['sample-006936.wav']

In [ ]:
with open('deepxi_trans1.pickle', 'wb') as e:
  pickle.dump(trans_dict, e, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

In [ ]:
wer = {}
with open('deepxi_trans1.pickle', 'rb') as e:
  wer = pickle.load(e)

In [ ]:
wer.keys()

In [ ]:
wer['short']['sample-006936.wav']